## PretrainDataset</br>
PretrainDataset 不是 PyTorch 或任何标准库中预定义的函数或类。它是一个约定俗成的名称，通常指代专门为大型语言模型（LLMs）等模型的预训练阶段设计的数据集类。</br>
这类数据集的特点是需要处理海量的文本数据，并根据预训练任务（如 Masked Language Modeling, Next Sentence Prediction 等）对数据进行特殊处理。</br>
PretrainDataset 通常是 torch.utils.data.Dataset 的子类。其核心原理是实现 __len__ 和 __getitem__ 方法。</br>
### 运行原理：</br>

懒加载 (Lazy Loading) / 流式处理： 大多数 PretrainDataset 不会一次性将所有数据读入内存。</br>相反，它们通常会：</br>
预处理： </br>将原始文本文件预处理成更易于加载的格式（如 jsonl 文件、二进制文件等），通常包含已标记化的 token IDs 和其他元数据。</br>
按需读取： </br>在 __getitem__ 方法被调用时，才从磁盘读取并处理单个或少量样本。</br>
任务定制化处理：</br> 在 __getitem__ 中，会根据当前预训练任务的需求对读取到的原始 token 序列进行进一步处理。</br>
### 例如：</br>
掩码语言模型 (MLM)： </br>随机选择一些 token 进行掩码（替换为 [MASK] token ID，或随机替换为其他 token，或保持不变），并记录被掩码的原始 token ID 作为目标。</br>
下一句预测 (NSP)： </br>从语料库中选取两个句子，判断它们是否是原文中的连续句子，并生成相应的标签和特殊的 [SEP] token。</br>
RoPE/位置编码： </br>虽然位置编码通常在模型 forward 中应用，但数据集可能会确保序列长度符合模型的要求，或者提供一些位置相关的元数据。</br>
缓存机制 (可选)： </br>为了提高效率，一些 PretrainDataset 可能会实现一个小的内存缓存，存储最近访问过的几个样本，避免频繁的磁盘 I/O。</br>
### 操作流程 (以常见的 MLM 任务为例)：
</br>
数据准备阶段 (离线/预处理)：
</br>
收集原始文本数据： </br>收集海量的文本语料（如维基百科、书籍、网页等）。</br>
分句/分段： </br>将文本分割成句子或段落。</br>
标记化 (Tokenization)： </br>
使用预训练模型对应的 Tokenizer（例如 BertTokenizer, GPT2Tokenizer 等）将文本转换为 token ID 序列。同时，添加特殊 token，如 [CLS], [SEP], [PAD]。</br>
序列化存储： </br>
将标记化后的 token ID 序列以及可能的元数据（如原始句子边界、文档边界）保存为高效的格式（如 TFRecord, HDF5, 或简单的 JSONL 文件）。</br>
这一步是关键，它避免了每次训练时都进行文本解析和标记化。</br>

### PretrainDataset 初始化 (__init__)：</br>

接收预处理后的数据文件路径、tokenizer、以及预训练任务相关的参数（如掩码概率 mlm_probability、最大序列长度 max_seq_length）。</br>
加载文件索引或元数据，但不加载实际数据到内存。</br>
__len__ 方法：</br>

返回数据集中总样本的数量（例如，处理后的文档或片段的数量）。</br>
__getitem__(idx) 方法 (核心)：</br>

读取原始数据：</br>
1. 根据 idx 从预处理文件中读取对应的 token ID 序列（可能是一个文档或两个相邻的段落）。</br>
2. 序列截断/填充： </br>
3. 根据 max_seq_length 对 token 序列进行截断或填充 ([PAD] token)。</br>
4. 应用预训练任务逻辑，比如以下几个常见的预训练任务逻辑要求：</br>
MLM 掩码： </br>
根据 mlm_probability 随机选择一些 token。对于这些 token，根据一定比例进行：</br>
替换为 [MASK] token ID。</br>
替换为随机 token ID。
保持不变。</br>
同时，生成一个 labels 序列，记录被掩码的原始 token ID，未被掩码的 token 处则用 -100（或其他忽略索引）标记。</br>
NSP 逻辑 ：</br>
随机选择一个真实下一句或一个随机非下一句。</br>
将它们拼接起来，用 [SEP] 分隔，并添加 [CLS] 在开头。</br>
生成一个 next_sentence_label (0 或 1)。</br>
生成 token_type_ids (Segment A 为 0，Segment B 为 1)。</br>
返回处理后的样本： 通常返回一个字典或元组，包含：</br>
input_ids: 经过掩码/拼接处理的 token ID 序列。</br>
attention_mask: 用于指示哪些 token 是真实内容，哪些是填充 ([PAD])。</br>
labels (MLM 目标): 记录被掩码的原始 token ID。</br>
token_type_ids (NSP 目标): 区分不同句子段。</br>
next_sentence_label (NSP 目标): 区分是否是下一句。</br>
torch.utils.data.DataLoader 包装：</br>

将 PretrainDataset 实例传递给 DataLoader，以便进行批处理、乱序、多线程加载等操作。</br>

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AutoTokenizer
import random

In [2]:
class MockTokenizer:
    def __init__(self):
        self.vocab = {"[PAD]": 0, "[CLS]": 1, "[SEP]": 2, "[MASK]": 3,
                      "hello": 4, "world": 5, "this": 6, "is": 7, "a": 8,
                      "test": 9, "sentence": 10, "another": 11, "example": 12,
                      "data": 13, "for": 14, "pretraining": 15, "model": 16}
        self.ids_to_tokens = {v: k for k, v in self.vocab.items()}
        self.pad_token_id = self.vocab["[PAD]"]
        self.cls_token_id = self.vocab["[CLS]"]
        self.sep_token_id = self.vocab["[SEP]"]
        self.mask_token_id = self.vocab["[MASK]"]

    def encode(self, text, add_special_tokens=False):
        # 简化版：直接将文本分割，并映射到ID
        tokens = text.lower().split()
        ids = [self.vocab.get(token, self.vocab["[MASK]"]) for token in tokens] # 未知词也用MASK代替
        return ids

    def convert_ids_to_tokens(self, ids):
        return [self.ids_to_tokens.get(id, "[UNK]") for id in ids]


In [3]:
tokenizer = MockTokenizer()
# 模拟预处理后的文本数据 (这里直接用ID表示，方便演示)
# 真实场景中，这些会从文件读取
raw_preprocessed_data = [
    tokenizer.encode("Hello world this is a test sentence"),
    tokenizer.encode("Another example for data pretraining"),
    tokenizer.encode("This is another test sentence for model"),
    tokenizer.encode("Hello world this is an example for pretraining"),
    tokenizer.encode("A test data for a model"),
    tokenizer.encode("Another sentence for the pretraining model"),
    tokenizer.encode("Example data for the test model"),
    tokenizer.encode("This is a world of pretraining"),
]

print("Simulated Raw Preprocessed Data (Token IDs):")
for item in raw_preprocessed_data:
    print(item)

Simulated Raw Preprocessed Data (Token IDs):
[4, 5, 6, 7, 8, 9, 10]
[11, 12, 14, 13, 15]
[6, 7, 11, 9, 10, 14, 16]
[4, 5, 6, 7, 3, 12, 14, 15]
[8, 9, 13, 14, 8, 16]
[11, 10, 14, 3, 15, 16]
[12, 13, 14, 3, 9, 16]
[6, 7, 8, 5, 3, 15]


In [4]:
class SimplePretrainDataset(Dataset):
    def __init__(self, tokenized_data, tokenizer, max_seq_length=None, mlm_probability=0.15):
        self.tokenized_data = tokenized_data
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.mlm_probability = mlm_probability

        # 确保tokenizer有必要的特殊token ID
        assert hasattr(tokenizer, 'pad_token_id')
        assert hasattr(tokenizer, 'cls_token_id')
        assert hasattr(tokenizer, 'sep_token_id')
        assert hasattr(tokenizer, 'mask_token_id')

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        # 1. 获取原始 token IDs
        input_ids = self.tokenized_data[idx]

        # 2. 添加特殊 token [CLS] 和 [SEP] (简化版，通常在预处理时完成，或在NSP任务中更复杂)
        # 比如： [CLS] + sentence_tokens + [SEP]
        input_ids = [self.tokenizer.cls_token_id] + input_ids + [self.tokenizer.sep_token_id]

        # 3. 截断或填充
        if self.max_seq_length:
            if len(input_ids) > self.max_seq_length:
                input_ids = input_ids[:self.max_seq_length]
            else:
                # 填充 [PAD] token
                padding_length = self.max_seq_length - len(input_ids)
                input_ids = input_ids + [self.tokenizer.pad_token_id] * padding_length

        # 4. 生成 attention_mask (1表示真实token，0表示填充token)
        attention_mask = [1] * len(input_ids)
        if self.max_seq_length and len(input_ids) < self.max_seq_length:
            attention_mask = attention_mask + [0] * (self.max_seq_length - len(attention_mask))

        # 5. 执行 MLM (Masked Language Model) 掩码
        # labels 复制 input_ids，然后将未被掩码的位置设为 -100 (被PyTorch的CrossEntropyLoss忽略)
        labels = list(input_ids) # 复制一份作为标签
        masked_indices = self._mask_tokens(input_ids, labels) # 修改 input_ids 和 labels

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long)
        }

    def _mask_tokens(self, inputs, labels):
        """
        根据 mlm_probability 掩码 tokens，并更新 labels。
        这部分逻辑通常更复杂，以符合BERT等模型的掩码策略 (80% MASK, 10% 随机, 10% 不变)。
        这里是一个简化版本。
        """
        masked_indices = []
        for i, token_id in enumerate(inputs):
            # 不掩码特殊 token
            if token_id in [self.tokenizer.cls_token_id, self.tokenizer.sep_token_id, self.tokenizer.pad_token_id]:
                continue

            # 随机决定是否掩码
            if random.random() < self.mlm_probability:
                masked_indices.append(i)
                # 80% 几率替换为 [MASK]
                if random.random() < 0.8:
                    inputs[i] = self.tokenizer.mask_token_id
                # 10% 几率替换为随机 token
                elif random.random() < 0.5: # 0.8 + 0.1 = 0.9，所以这里是 <0.5 for the remaining 0.2
                    inputs[i] = random.randint(0, len(self.tokenizer.vocab) - 1)
                # 10% 几率保持不变 (labels仍然是原始token)

                # 将 labels 中未被掩码的 token 设为 -100，表示在损失计算时忽略
            else:
                labels[i] = -100
        return masked_indices

# 4. 使用 PretrainDataset 和 DataLoader
max_seq_len = 20 # 示例最大序列长度
mlm_prob = 0.15 # 掩码概率

pretrain_dataset = SimplePretrainDataset(
    tokenized_data=raw_preprocessed_data,
    tokenizer=tokenizer,
    max_seq_length=max_seq_len,
    mlm_probability=mlm_prob
)

pretrain_dataloader = DataLoader(pretrain_dataset, batch_size=2, shuffle=True)

print(f"\nExample from DataLoader (batch_size=2, max_seq_len={max_seq_len}, mlm_probability={mlm_prob}):")
# 获取一个批次数据并打印
for i, batch in enumerate(pretrain_dataloader):
    if i == 0: # 只打印第一个批次
        print("Input IDs (masked):")
        print(batch["input_ids"])
        print("\nAttention Mask:")
        print(batch["attention_mask"])
        print("\nLabels (original token IDs for masked positions, -100 otherwise):")
        print(batch["labels"])

        # 转换为可读的tokens（仅用于演示）
        print("\nInput Tokens (masked):")
        for sample_ids in batch["input_ids"]:
            print(tokenizer.convert_ids_to_tokens(sample_ids.tolist()))
        print("\nLabel Tokens:")
        for sample_labels in batch["labels"]:
            # 过滤掉 -100
            readable_labels = [tokenizer.ids_to_tokens.get(id.item(), "[UNK]") if id.item() != -100 else -100 for id in sample_labels]
            print(readable_labels)
        break


Example from DataLoader (batch_size=2, max_seq_len=20, mlm_probability=0.15):
Input IDs (masked):
tensor([[ 1, 12, 13, 14,  3,  9, 16,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [ 1,  3, 12, 14, 13, 15,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])

Attention Mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

Labels (original token IDs for masked positions, -100 otherwise):
tensor([[   1, -100, -100, -100,    3, -100, -100,    2,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   1,   11, -100, -100, -100, -100,    2,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])

Input Tokens (masked):
['[CLS]', 'example', 'data', 'for', '[MASK]', 'test', 'model', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P